In [0]:
import numpy as np
import tensorflow as tf

In [0]:
def boardtostate(squares):
	id1 = 0
	for i in range(9):
		val = 0
		if (squares[i]!='-'):
			if squares[i] == 'X':
				val = 1
			else:
				val = 2
			id1 += val * (3**i)
	return id1

In [0]:
def statetoboard(s):
	state = s
	board = ''
	for i in range(9):
		val = int(s/(3**(8-i)))
		if val!=0:
			if val == 1:
				board += 'X'	# X => 1
			else:
				board += 'O'	# O => 2
			s -= (int) (val * (3**(8-i)))
		elif val==0:
			board += '-'
	return board

In [0]:
def Actions(state):
	existing = state
	actions = []
	for i in range(9):
		val = (int) (state/(3**(8-i)))
		if val == 0:
			sp = existing + (int) (2*(3**(8-i)))  #agent is'O' so "2"
			actions.append(sp)
		state-= (int) (val * (3**(8-i)))
	return actions

In [0]:
squares = 'X--OX---O'

In [43]:
Actions(13728)

[18102, 15186, 13730]

In [0]:
state_size = 1
action_size = np.array(Actions(state)).size

max_episodes = 200
learning_rate = 0.05
gamma = 0.9

In [0]:
def discount_and_normalize_rewards(episode_rewards):
  discounted_episode_rewards = np.zeros_like(episode_rewards)
  cumulative = 0.0
  for i in reversed(range(len(episode_rewards))):
    cumulative = cumulative * gamma + episode_rewards[i]
    discounted_episode_rewards[i] = cumulative
  
  mean = np.mean(discounted_episode_rewards)
  std = np.std(discounted_episode_rewards)
  discounted_episode_rewards = (discounted_episode_rewards-mean)/std
  
  return discounted_episode_rewards

I am creating 3 layers here (Input -> FC1 -> FC2 -> FC3 -> Output)

Ouput function is softmax that makes the outputs to a probability distribution.

In [0]:
with tf.name_scope(name="input"):
  
  input_ = tf.placeholder(tf.float32,[None,1],name="input")
  actions = tf.placeholder(tf.int32, [None,action_size],name="actions")
  discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,],name="discounted_episode_rewards_")
  
  mean_reward_ = tf.placeholder(tf.float32,name="mean_reward")
  
  with tf.name_scope("FC1"):
    FC1 = tf.contrib.layers.fully_connected(inputs=input_,num_outputs=10,activation_fn=tf.nn.relu,weights_initializer=tf.contrib.layers.xavier_initializer())
  
  with tf.name_scope("FC2"):
    FC2 = tf.contrib.layers.fully_connected(inputs=FC1,num_outputs=action_size,activation_fn=tf.nn.relu,weights_initializer=tf.contrib.layers.xavier_initializer())
  
  with tf.name_scope("FC3"):
    FC3 = tf.contrib.layers.fully_connected(inputs=FC2,num_outputs=action_size,activation_fn=None,weights_initializer=tf.contrib.layers.xavier_initializer())
    
  with tf.name_scope("softmax"):
    action_distribution = tf.nn.softmax(FC3)
  
  with tf.name_scope("loss"):
    neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=FC3,labels=actions)
    loss = tf.reduce_mean(neg_log_prob*discounted_episode_rewards_)
  
  with tf.name_scope("train"):
    train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

As we had to maximize the Rewards but here we are using gradient descent algorithm because we have taken negative of log , so minimizing this function will actually maximize the rewards that is gradient ascent.

In [0]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]

saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for episode in range(max_episodes):
    episode_rewards_sum = 0
    state = 0
    while True:
      action_probability_distribution = sess.run(action_distribution,feed_dict={input_:state.reshape([1,1])})
      action = np.random.choice(range(action_probability_distribution.shape[1]),p= action_probability_distribution.ravel())   # p= probability array and .ravel() -> to contiguous flatten the array
      
      #perform the action
      new_state, reward, performed = ### to implement    After performing the action on environment it should return new_state-> int, reward -> int, performed -> bool
      
      episode_states.append(state)
      action_ = np.zeros(action_size)
      action_[action] = 1
      episode_actions.append(action_)
      episode_rewards.append(reward)
      
      if performed: #If action performed
        episode_rewards_sum = np.sum(episode_rewards)
        allRewards.append(episode_rewards_sum)
        total_rewards = np.sum(allRewards)
        mean_reward = np.divide(total_rewards,episode+1)
        maximumRewardRecorded = np.amax(allRewards)
        print("**************")
        print("Episode:",episode)
        print("Reward: ",episode_rewards_sum)
        print("Mean Reward: ", mean_reward)
        print("Max Reward so far: ", maximumRewardRecorded)
        print("**************")
        
        discounted_episode_rewards = discount_and_normalize_rewards(episode_rewards)
        loss_,_ = sess.run([loss,train_opt],feed_dict={input_: np.vstack(np.array(episode_states)),
                                                       actions: np.vstack(np.array(episode_actions)),
                                                       discounted_episode_rewards_: discounted_episode_rewards 
                                                      })
        summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(episode_states)),
                                                                 actions: np.vstack(np.array(episode_actions)),
                                                                 discounted_episode_rewards_: discounted_episode_rewards,
                                                                    mean_reward_: mean_reward
                                                                })
                
               
        writer.add_summary(summary, episode)
        writer.flush()
        episode_states, episode_actions, episode_rewards = [],[],[]
        break
    state = new_state
  
  if episode % 50 == 0:
    saver.save(sess,".models/model.ckpt")
    print("SAVED")